<a href="https://colab.research.google.com/github/boytjj/taojingjun/blob/master/Shared_GanSpace_Exploration_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Script adopted from Derrich (Artificial Images YouTube Channel)**


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
%mkdir /content/styleganiitb
%cd /content/styleganiitb

mkdir: cannot create directory ‘/content/styleganiitb’: File exists
/content/styleganiitb


In [ ]:
!git clone https://github.com/skyflynil/stylegan2


/content/drive/My Drive/styleganiitb
Cloning into 'stylegan2'...
remote: Enumerating objects: 219, done.
remote: Total 219 (delta 0), reused 0 (delta 0), pack-reused 219
Receiving objects: 100% (219/219), 2.29 MiB | 4.03 MiB/s, done.
Resolving deltas: 100% (120/120), done.
/content/drive/My Drive/styleganiitb


**Change path_to_model and path_to_components to shared files**

In [119]:
path_to_tf_stylegan = '/content/stylegan2'
path_to_model = '/content/drive/MyDrive/network-snapshot-010162.pkl'
path_to_components = '/content/drive/MyDrive/stylegan2-mmfloorplan_style_ipca_c20_n1000000_w.npz'
named_directions = {} #init named_directions dict to save directions

In [120]:
#@title Load model { display-mode: "both" }

#@markdown This code loads the tensorflow version of StyleGAN2
%cd $path_to_tf_stylegan
import tensorflow as tf
import ipywidgets as widgets
import pretrained_networks
import PIL.Image
import numpy as np
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import IPython.display
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio
import pretrained_networks

src_model = path_to_model

_G, _D, Gs = pretrained_networks.load_networks(src_model)
Gs_syn_kwargs = dnnlib.EasyDict()
batch_size = 1
Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
Gs_syn_kwargs.randomize_noise = True
Gs_syn_kwargs.minibatch_size = batch_size

noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

/content/stylegan2


In [121]:
component_value = 9
#component_value = 'random'

comps = np.load(path_to_components)
lst = comps.files
latent_dirs = []
latent_stdevs = []

load_activations = True

for item in lst:
    # loading latent directions
    # TODO: what is the difference btwn act_comp and latent_comp? should I use one, the other or both?

    # print('\n')
    # print('--------'*10)
    # print(item)
    if load_activations:
      if item == 'act_comp':
        # print(comps[item].shape[0])
        for i in range(comps[item].shape[0]):
          # print(comps[item][i].shape)
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        # print()
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        # print(comps[item].shape[0])
        for i in range(comps[item].shape[0]):
          # print(comps[item][i].shape)
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        # print()
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
        
    # print('--------'*10)
    # print('\n')
    

#load one at random 
if component_value == 'random':
  num = np.random.randint(num_components)
else:
  num = component_value

if num in named_directions.values():
  print(f'Direction already named: {list(named_directions.keys())[list(named_directions.values()).index(num)]}')

random_dir = latent_dirs[num]
random_dir_stdev = latent_dirs[num]

random_dir = np.expand_dims(random_dir, axis=0)
#TODO: I think currently this will only work for 1024x1024 models? make this adjustable but idk the rule
# for 512 x 512 models it's 16, 1024 x 1024 = 18
n = 18
if Gs.input_shape[1:][0] == 512:
  n = 18
random_dir = np.tile(random_dir, [1, n, 1]) 

print(f'Loaded Component No. {num}')


Loaded Component No. 9


Define a name in Name component box and use in next code block as direction name


In [123]:
#@title Run UI
from IPython.utils import io

def name_direction(sender):
  if num in named_directions.values():
    target_key = list(named_directions.keys())[list(named_directions.values()).index(num)]
    print(f'Direction already named: {target_key}')
    print(f'Overwriting... ')
    del(named_directions[target_key])
  named_directions[text.value] = num
  save_direction(random_dir, text.value)
  for item in named_directions:
    print(item, named_directions[item])

def save_direction(direction, filename):
  filename += ".npy"
  np.save(filename, direction, allow_pickle=True, fix_imports=True)
  print(f'Latent direction saved as {filename}')

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def generate_mov(seed, truncation, direction_vec, scale, n_frames, out_name = 'out', noise_spec = None, loop=True):
  """Generates a mov moving back and forth along the chosen direction vector"""
  # Example of reading a generated set of images, and storing as MP4.
  %mkdir out
  movieName = f'out/{out_name}.mp4'
  offset = -10
  step = 20 / n_frames
  imgs = []
  for i in range(n_frames):
    print(f'{i} / {n_frames}')
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation is not None:
        Gs_kwargs.truncation_psi = truncation
            
    rnd = np.random.RandomState(seed)
        
    if noise_spec is None:
      tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
    else:
      tflib.set_vars(noise_spec)
    
    batch_size = 1
    all_seeds = [seed] * batch_size
    all_z = np.stack([np.random.RandomState(seed).randn(*Gs.input_shape[1:]) for seed in all_seeds]) # [minibatch, component]
    all_w = Gs.components.mapping.run(all_z, None) # [minibatch, layer, component]
    if truncation != 1:
        w_avg = Gs.get_var('dlatent_avg')
        all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]
    all_w += direction_vec * offset * scale
    all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs)
    #save image and display
    final_im = PIL.Image.fromarray(np.median(all_images, axis=0).astype(np.uint8))
    imgs.append(final_im)
    #increase offset
    offset += step
  if loop:
    imgs += imgs[::-1]
  with imageio.get_writer(movieName, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(np.array(image))

def display_sample(seed, truncation, distance, scale, disp=True, save=None, noise_spec=None):
    # blockPrint()
    with io.capture_output() as captured:
      # weighted_average(Gs, Gsd, blending)
      
      Gs_kwargs = dnnlib.EasyDict()
      Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
      Gs_kwargs.randomize_noise = False
      if truncation is not None:
          Gs_kwargs.truncation_psi = truncation
          
      rnd = np.random.RandomState(seed)
      
      if noise_spec is None:
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
      else:
        tflib.set_vars(noise_spec)
      
      batch_size = 1
      all_seeds = [seed] * batch_size
      all_z = np.stack([np.random.RandomState(seed).randn(*Gs.input_shape[1:]) for seed in all_seeds]) # [minibatch, component]
      all_w = Gs.components.mapping.run(all_z, None) # [minibatch, layer, component]
      if truncation != 1:
          w_avg = Gs.get_var('dlatent_avg')
          all_w = w_avg + (all_w - w_avg) * truncation # [minibatch, layer, component]
      print(random_dir)
      print(distance)
      print(scale)
      all_w += random_dir * distance * scale
      all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs)
      #save image and display
      final_im = PIL.Image.fromarray(np.median(all_images, axis=0).astype(np.uint8)).resize((400,400),Image.LANCZOS)
    if disp:
      # print('no display')
      display(final_im)
    if save is not None:
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')

seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=5, step=0.05, value=1, description='Scale: ', continuous_update=False)
text = widgets.Text(description="Name component here", style=style, width=200)

bot_box = widgets.HBox([seed, truncation, distance, scale, text])
ui = widgets.VBox([bot_box])

out = widgets.interactive_output(display_sample, {'seed': seed, 'truncation': truncation, 'distance': distance, 'scale': scale})

display(ui, out)
text.on_submit(name_direction)




Output()

In [ ]:
#script to generate a movie moving back and forth along the direction

direction_name = 'com9'
loc = named_directions[direction_name]
num_samples = 1
#num_samples = 20

for i in range(num_samples):
  #s = np.random.randint(0, 10000)
  s = 2
  generate_mov(seed = s, truncation = 0.8, direction_vec = latent_dirs[loc], scale = 1, n_frames = 50, out_name = f'{direction_name}_{i}', loop=True)

mkdir: cannot create directory ‘out’: File exists
0 / 50
1 / 50
2 / 50
3 / 50
4 / 50
5 / 50
6 / 50
7 / 50
8 / 50
9 / 50
10 / 50
11 / 50
12 / 50
13 / 50
14 / 50
15 / 50
16 / 50
17 / 50
18 / 50
19 / 50
20 / 50
21 / 50
22 / 50
23 / 50
24 / 50
25 / 50
26 / 50
27 / 50
28 / 50
29 / 50
30 / 50
31 / 50
32 / 50
33 / 50
34 / 50
35 / 50
36 / 50
37 / 50
38 / 50
39 / 50
40 / 50
41 / 50
42 / 50
43 / 50
44 / 50
45 / 50
46 / 50
47 / 50
48 / 50
49 / 50


Extra
